In [260]:
import pandas as pd
import nltk
import os
import re

nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Rowjak\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Rowjak\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [261]:
path = os.getcwd()

In [262]:
df_train = pd.read_parquet(os.path.join(path, "dataset/train_combined.parquet"))

In [263]:
# df_test = pd.read_parquet(os.path.join(path, "dataset/test_combined.parquet"))
# df_dev = pd.read_parquet(os.path.join(path, "dataset/dev_combined.parquet"))
# df_train_sample = pd.read_parquet(os.path.join(path, "dataset/train_sample_combined.parquet"))
# df_xtreme_dev = pd.read_parquet(os.path.join(path, "dataset/xtreme_dev_combined.parquet"))
# df_xtreme_test = pd.read_parquet(os.path.join(path, "dataset/xtreme_test_combined.parquet"))

In [264]:
# df_train.info

#Dataset yang digunaakn

In [265]:
def extract_article(sentences):
    # Gabungkan array kata-kata menjadi kalimat
    text = ' '.join([' '.join(sentence) for sentence in sentences])
    # Hilangkan spasi sebelum tanda baca seperti .,;:!?()
    text = re.sub(r'\s+([.,:;!?()])', r'\1', text)
    # Hilangkan spasi setelah tanda baca '.' jika huruf setelahnya kecil
    text = re.sub(r'\.\s+([a-z])', r'.\1', text)
    text = re.sub(r'\(\s+', '(', text)  # Hilangkan spasi setelah '('
    text = re.sub(r'\s+\)', ')', text)  # Hilangkan spasi sebelum ')'
    text = re.sub(r'\[\s+', '[', text)  # Hilangkan spasi setelah '['
    text = re.sub(r'\s+\]', ']', text)  # Hilangkan spasi sebelum ']'
    return text

def extract_sentences_from_index(clean_article, extractive_summary):
    return [clean_article[i] for i in extractive_summary]

def convert_time_to_minutes(waktu):
    waktu = re.sub(r'\s*WIB|\s*WIT|\s*WITA', '', waktu)
    time_obj = pd.to_datetime(waktu, format='%H:%M')
    return time_obj.hour * 60 + time_obj.minute

def preprocessing_article(text):
    # ubah semua menjadi huruf kecil
    text = text.lower()
    # ubah kata yang berulang (dengan tanda penghubung) menjadi terpisah
    text = re.sub(r'\b(\w+)-\1\b', r'\1 \1', text)
    # hilangkan teks yang memiliki tanda kurun () dan []
    text = re.sub(r'\(.*?\)', '', text)
    text = re.sub(r'\[.*?\]', '', text)
    # hilangkan awalan liputan6.com kemudian setelah itu tanda :
    text = re.sub(r'^liputan6\.com, [^:]+: ', '', text)
    # hilangkan semua tanda baca
    text = re.sub(r'[^\w\s]', '', text)
    # hilangkan spasi berlebih
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def remove_stopwords(text):
    text = re.sub(r'\b\d{1,2}:\d{2}\s?(WIB|WIT|WITA)?\b', '', text)
    stop_words = set(stopwords.words('indonesian'))
    words = text.split()
    filtered_words = [word for word in words if word not in stop_words]
    return ' '.join(filtered_words)

In [266]:
def save_preprocessing(df_used, df_save_path):
    df_used['ext_clean_article'] = df_used['clean_article'].apply(extract_article)
    df_used['ext_clean_summary'] = df_used['clean_summary'].apply(extract_article)
    df_used['extractive_summary_sentences'] = df_used.apply(lambda row: extract_sentences_from_index(row['clean_article'], row['extractive_summary']), axis=1)
    df_used['ext_extractive_summary'] = df_used['extractive_summary_sentences'].apply(extract_article)
    df_used['prep_clean_article'] = df_used['ext_clean_article'].apply(preprocessing_article)
    df_used['prep_clean_summary'] = df_used['ext_clean_summary'].apply(preprocessing_article)
    df_used['prep_extractive_summary'] = df_used['ext_extractive_summary'].apply(preprocessing_article)
    df_used['prep_clean_article_no_sw'] = df_used['prep_clean_article'].apply(remove_stopwords)
    df_used.to_parquet(df_save_path)

In [267]:
# save_preprocessing(df_test,"dataset/used/test_prep.parquet")
# save_preprocessing(df_dev,"dataset/used/dev_prep.parquet")
# save_preprocessing(df_xtreme_dev,"dataset/used/xtreme_dev_prep.parquet")
# save_preprocessing(df_xtreme_test,"dataset/used/xtreme_test_prep.parquet")
# save_preprocessing(df_train_sample,"dataset/used/train_sample_prep.parquet")

In [268]:
save_preprocessing(df_train,"dataset/used/train.parquet")